Layer Normalization (LayerNorm) 是一种归一化技术，常用于深度学习模型中，特别是在 Transformer 模型中。


## 1. 与 Batch Normalization 的区别

- **Batch Normalization**：在批次（batch）维度上归一化，依赖于 batch size，适合图像等任务。
- **Layer Normalization**：在特征（feature）维度上归一化，对每个样本单独处理，不依赖 batch size，适合 NLP、Transformer 等序列任务。

---

## 2. 计算流程

假设输入 $x$ 的 shape 为 $(batch, ..., features)$，LayerNorm 针对最后一个维度（features）做归一化。

1. **计算均值和方差**  
   对每个样本的特征维度，计算均值 $\mu$ 和方差 $\sigma^2$：
   $$
   \mu = \frac{1}{d} \sum_{i=1}^d x_i
   $$
   $$
   \sigma^2 = \frac{1}{d} \sum_{i=1}^d (x_i - \mu)^2
   $$
   其中 $d$ 是特征维度大小。

2. **标准化**  
   $$
   \hat{x}_i = \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}}
   $$
   其中 $\epsilon$ 是一个很小的正数，防止除以零。

3. **缩放和平移**  
   $$
   y_i = \gamma \hat{x}_i + \beta
   $$
   其中 $\gamma$ 和 $\beta$ 是可学习参数（与特征维度同形状），用于恢复模型表达能力。

---

In [ ]:
import torch
import torch.nn as nn
 
class LayerNorm(nn.Module):
    def __init__(self,num_features,eps=1e-6):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))
        self.eps = eps
 
    def forward(self,x):
        mean = x.mean(dim=-1,keepdim=True)
        std = x.std(dim=-1,keepdim=True,unbiased=False)
        normalized_x = (x - mean) / (std + self.eps)
        return self.gamma * normalized_x + self.beta
 
if __name__ == '__main__':
    batch_size = 2
    seqlen = 3
    hidden_dim = 4
 
    # 初始化一个随机tensor
    x = torch.randn(batch_size,seqlen,hidden_dim)
    print(x)
 
    # 初始化LayerNorm
    layer_norm  = LayerNorm(num_features=hidden_dim)
    output_tensor = layer_norm(x)
    print("output after layer norm:\n,",output_tensor)
 
    torch_layer_norm = torch.nn.LayerNorm(normalized_shape=hidden_dim)
    torch_output_tensor = torch_layer_norm(x)
    print("output after torch layer norm:\n",torch_output_tensor)

RMS Norm 
RMSNorm层则是通过计算沿着最后一个维度的均方根来归一化输入

并使用可学习的权重向量对归一化后的结果进行缩放。



In [ ]:
class RMSNorm(nn.Module):
    def __init__(self, num_features, eps=1e-8):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(num_features))
        self.eps = eps

    def forward(self, x):
        # 计算均方根
        rms = x.pow(2).mean(dim=-1, keepdim=True).add(self.eps).sqrt()
        x_norm = x / rms
        return self.weight * x_norm

# 示例：对输入张量应用 RMSNorm
rms_norm = RMSNorm(num_features=hidden_dim)
rms_output = rms_norm(x)
print("output after RMSNorm:\n", rms_output)

## DeepNorm

DeepNorm 是对 LayerNorm 的一种改进归一化方法，主要用于极深的 Transformer 网络（如 1000 层以上），显著提升了深层模型的稳定性和训练能力。其核心思想是通过缩放残差分支和参数初始化，缓解深层网络训练中的梯度消失或爆炸问题。

### 主要改进点

1. **输入放大（残差缩放）**  
   - 在每个子层（如注意力或前馈网络）输入 LayerNorm 之前，先将输入乘以一个放大系数 $\alpha$（$\alpha > 1$）。
   - 这样可以增强残差分支的信号，防止深层网络中信号衰减。

2. **参数初始化调整**  
   - 在 Xavier 初始化时，缩小部分参数的初始化范围，使深层网络更稳定。

### DeepNorm 的表达式

对于第 $l$ 层的子层，DeepNorm 的残差连接公式为：

$$
\text{Output}_l = \text{LayerNorm}(\alpha \cdot x + F(x))
$$

- 其中 $x$ 是输入，$F(x)$ 是子层（如注意力或前馈网络）的输出，$\alpha$ 是放大系数。
- $\alpha$ 的取值通常根据 Encoder 层数 $N$ 和 Decoder 层数 $M$ 计算，例如：
  - 对于 Encoder: $\alpha = (3N)^{1/4}$
  - 对于 Decoder: $\alpha = (3M)^{1/4}$

### 作用

- 通过放大输入和调整初始化，DeepNorm 能有效缓解深层 Transformer 的训练难题，使网络层数可以扩展到 1000 层以上。
- 适用于极深的 Encoder-Decoder 架构（如 DeepNet）。


In [ ]:
class DeepNorm(nn.Module):
    def __init__(self, num_features, alpha=1.0, eps=1e-6):
        super().__init__()
        self.alpha = alpha  # DeepNorm放大输入的系数
        self.layer_norm = LayerNorm(num_features, eps=eps)

    def forward(self, x):
        # DeepNorm: 先放大输入，再做LayerNorm
        x_scaled = x * self.alpha
        return self.layer_norm(x_scaled)

# 示例：对输入张量应用 DeepNorm
# alpha 通常根据网络层数设置，这里举例设为1.5
deep_norm = DeepNorm(num_features=hidden_dim, alpha=1.5)
deep_output = deep_norm(x)
print("output after DeepNorm:\n", deep_output)

Post Norm结构模型中warm up如何起作用的？
warmup是Transformer训练的关键步骤，没有它可能不收敛，或者收敛到比较糟糕的位置。为什么会这样呢？
warmup是在训练开始阶段，将学习率从0缓增到指定大小，而不是一开始从指定大小训练。如果不进行warmup，那么模型一开始就快速地学习，由于梯度消失，模型对越靠后的层越敏感，也就是越靠后的层学习得越快，然后后面的层是以前面的层的输出为输入的，前面的层根本就没学好，所以后面的层虽然学得快，但却是建立在糟糕的输入基础上的。很快地，后面的层以糟糕的输入为基础到达了一个糟糕的局部最优点，此时它的学习开始放缓（因为已经到达了它认为的最优点附近），同时反向传播给前面层的梯度信号进一步变弱，这就导致了前面的层的梯度变得不准。

所以，如果Post Norm结构的模型不进行warmup，能观察到的现象往往是：loss快速收敛到一个常数附近，然后再训练一段时间，loss开始发散，直至NAN。

如果进行warmup，那么留给模型足够多的时间进行“预热”，在这个过程中，主要是抑制了后面的层的学习速度，并且给了前面的层更多的优化时间，以促进每个层的同步优化。

简要介绍各种Normalization method
LayerNorm：LayerNorm会计算当前Layer的所有激活值的均值μ和方差σ，然后对激活值X减去均值μ，除以方差σ，再通过可训练的缩放参数 γ 进行缩放，最后添加可训练的平移参数β 得到 Y。LN最重要的两个部分是平移不变性和缩放不变性。

LayerNorm规范化activations的第一动量均值(mean)和第二动量方差(variance)。



RMSNorm：RMSNorm是改进归一化方法的LayerNorm。相比LayerNorm中利用均值和方差进行归一化，RMSNorm 利用均方根进行归一化。RMSNorm会计算当前Layer的所有激活值的均方根rms,然后对激活值X除以均方根rms，再通过可训练的缩放参数 γ 进行缩放，最后得到 Y。

RMSNorm规范化activations的第二动量均方根(RMS)。RMSNorm的收敛速度比LN要快很多。



DeepNorm：与Post-LN相比，DeepNorm在LayerNorm之前对残差链接进行up-scale，在初始化阶段down-scale模型参数。DeepNorm兼具Pre-LN的训练稳定和Post-LN的效果性能。需要注意的是，该方法只会扩大前馈网络的权值的规模，以及attention层的投影值。

DeepNorm试图结合LN和RMSNorm长处，同时规范化activations的第一动量和第二动量。



![image.png](./static/accesswximg.png)
![image.png](./static/ln.png)
BatchNorm：对每一个batch进行操作，使得对于这一个batch中所有的输入数据，它们的每个特征都是均值为0，方差为1的分布。在BN后，需要再加一个线性变换操作，让数据恢复其表达能力(让模型学习参数γ 和 β)。

LayerNorm：整体做法类似于BN，不同的是LN不是在特征间进行标准化操作（横向操作），而是在整条数据间进行标准化操作（纵向操作）。



BN和LN的区别：BN和LN的作用对象不同，BatchNorm认为相同维的特征具有相同分布，在特征维度上开展归一化操作，归一化的结果保持样本之间的可比较性。而LayerNorm认为每个样本内的特征具有相同分布，因此针对每一个样本进行归一化处理，保持相同样本内部不同对象的可比较性。

